In [1]:
import requests
import time
import pandas as pd
import bs4
import sqlite3
import csv
import matplotlib.pyplot as plt

def crawler():
    global dateTag
    global textTag
    global titleTag 
    
    dateTag = objSoup.select('div.post-date')
    textTag = objSoup.select('div.entity-list-body')
    #ContentEle=objSoup.select("div.view-content")
    titleTag=objSoup.find_all('h3',class_="entity-list-title")
    print("已使用爬蟲取得資料")
def toCSV():
    global csvFile
    for title in titleTag:
        content.append(title.get_text())
    #for text in objSoup.find_all('div',class_="entity-list-body"):    
    for textTemp in textTag: 
        text_Store.append(textTemp.text)
        if len(text_Store)>=10:
            break
    #for date in objSoup.find_all('div',class_="post-date"):    
    for date in dateTag:    
        date_Store.append(date.text)  
    addTag ={'Title':content,'Text':text_Store,'Date':date_Store}
    csvFile = pd.DataFrame(addTag)
    csvFile.to_csv('News.csv',encoding='utf_8_sig')
    print("已把爬蟲資料儲存為News.csv")
    
def printCSV():
    print(csvFile)
def print_text(): 
    for i in range(0,10):
        print("\n事實查核報告: " + titleTag[i].text)# 列出標題
        print("\n",textTag[i].text) # 列出内容
        print("\n",dateTag[i].text) # 列出日期
        print("========================================================================================")
        
def load_Data():
    print("\n查詢資料庫裏的資料")
    results = conn.execute("SELECT * from News")
    for record in results:
        print("========================================================================================")
        print("Index = ", record[0])
        print("\n標題 = ", record[1])
        print("\n内容 = ", record[2])
        print("\n日期 = ", record[3])
        

def delete_sql():
    sql = '''DROP TABLE IF EXISTS News'''    
    conn.execute(sql)  
    sql = '''Create table News(IndexNum int,Title text,Text text,Date text)'''
    conn.execute(sql)   
    print("已刪除資料庫裏的資料")
def updateDB():
    fn = 'News.csv'
    with open(fn,encoding="utf-8") as csvFile: # 儲存在SQLite
        csvReader = csv.reader(csvFile)
        listCsv = list(csvReader) # 轉成串列
        csvData = listCsv[1:]  # 切片刪除前1 rows
        for row in csvData:
            IndexSQL = row[0] # index
            titleSQL = row[1] # 標題
            textSQL = row[2]  #内容
            dateSQL = row[3] # 日期
            x = (IndexSQL,titleSQL,textSQL, dateSQL)
            sql = '''insert into News(IndexNum,Title,Text,Date) values(?,?,?,?)'''
            conn.execute(sql,x)
            conn.commit()
        print("已更新資料庫裏的資料")
def closeDB():
    conn.close() # 關閉資料庫連線
#===================================Main=================================================================    
conn = sqlite3.connect("Group.db")
conn.execute('PRAGMA encoding="UTF-8";')

date_Store = []
text_Store = []
content = []

response = requests.get('https://tfc-taiwan.org.tw/articles/report')
response.encoding='utf-8'

objSoup = bs4.BeautifulSoup(response.text, 'lxml')


while True:
    print("\n======================假新聞判斷系統========================")
    print("1.爬蟲台灣事實查核中心的資料")
    print("2.輸出爬蟲的資料")
    print("3.把爬蟲資料儲存為CSV檔")
    print("4.輸出News.csv裏的資料")
    print("5.刪除資料庫裏現有的資料")
    print("6.更新資料庫裏的資料")
    print("7.查詢資料庫裏的資料")
    print("8.Exit")
    print("=============================================================")
    menu_input = int(input("\n請選擇要使用的功能(1~8) :"))
    if menu_input==1:
        
        crawler()
        
    elif menu_input==2:
        try:
            print_text()
        except:
            print("Error!,需先用爬蟲取得資料(選項1)")
            
    elif menu_input==3:
        try:
            toCSV()
        except:
            print("Error!,需先用爬蟲取得資料(選項1)")
            
    elif menu_input==4:
        try:
            printCSV()
        except:
            print("Error!,需先把爬蟲資料儲存為CSV檔(選項3)")
            
    elif menu_input==5:
        delete_sql()
    
    elif menu_input==6:
        try:
            updateDB()
        except:
            print("Error!,需先把爬蟲資料儲存為CSV檔(選項3)")
            
    elif menu_input==7:
        try:
            load_Data()
        except:
            print("Error!,需先把爬蟲資料儲存為CSV檔(選項3)")
            
    elif menu_input==8:
        closeDB()
        print("Exit")
        break


======================假新聞判斷系統========================
1.爬蟲台灣事實查核中心的資料
2.輸出爬蟲的資料
3.把爬蟲資料儲存為CSV檔
4.輸出News.csv裏的資料
5.刪除資料庫裏現有的資料
6.更新資料庫裏的資料
7.查詢資料庫裏的資料
8.Exit


KeyboardInterrupt: Interrupted by user